In [1]:
import s3fs
import xarray as xr
from toolz import partition


In [2]:
CONSISTENT_VARS = {
    "time",
    "lat",
    "lon",
    "dt_analysis",
    "l2p_flags",
    "quality_level",
    "satellite_zenith_angle",
    "sea_surface_temperature",
    "sses_bias",
    "sses_count",
    "sses_standard_deviation",
    "sst_dtime",
}

In [3]:
source_path = "s3://imos-data/IMOS/SRS/SST/ghrsst/L3S-1d/day/2022/*.nc"
store_path = "s3://imos-data-pixeldrill/alexl/sst_allatonce/"

In [4]:
s3 = s3fs.S3FileSystem(anon=False)
files = s3.glob(source_path)

In [7]:
first_file = True
completed = 0
for files_subset in partition(10, files):
    paths = [s3.open(file) for file in files_subset]
    ds = xr.open_mfdataset(paths, engine="h5netcdf", chunks={"time": 10, "lon": 1000, "lat": 1000})
    available_vars = set(ds.variables)
    to_drop_vars = list(available_vars - CONSISTENT_VARS)
    ds = ds.drop_vars(to_drop_vars)

    # ds.chunks = {"time": 10, "lon": 1000, "lat": 1000}

    if first_file:
        ds.to_zarr(store_path, mode="w", consolidated=True)
        first_file = False
    else:
        ds.to_zarr(store_path, mode="a", append_dim="time", consolidated=True)

